COMPETITION WEBSITE: https://www.kaggle.com/c/titanic

In [57]:
import pandas as pd

In [58]:
train_path = "./train.csv"
test_path = "./test.csv"

df = pd.read_csv(train_path)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [59]:
# drop the passenger id (we dont need it)
df.drop("PassengerId", axis=1, inplace=True)

# Preprocessing + EDA
dataset pattern or smthng, idk

In [60]:
# find the datatype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [61]:
# find the null
df.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [62]:
# find the imbalance
df.groupby("Survived").size()

Survived
0    549
1    342
dtype: int64

### Preprocessing pt.1
my plan is:
- eliminate name (we dont want the model to include name to calculate their survivalability)
- change sex to numerical (male = 1, female = 0)
- fill null in age, then change age into int
- find the purpose of ticket, cabin, embarked (i want to drop all this sht) (if cabin, embarked is important then fill the null)

#### Eliminate Name

In [63]:
# simple drop name
def remove_name(df):
    df.drop("Name", axis=1, inplace=True)
    return df

#### Change the sex into int

In [64]:
# change sex to int (1, 0)
def numeric_sex(df):
    df["Sex"].replace("male", 1, inplace=True)
    df["Sex"].replace("female", 0, inplace=True)
    return df

#### Fix the age

In [65]:
## fill the null with average age -> tried this but only get 0.77 score (fix in pt2)
# def fill_age(df):
#     df["Age"] = df["Age"].fillna(df["Age"].mean())
#     return df

#### ticket, embarked, and cabin

In [66]:
df.groupby("Ticket").size()

Ticket
110152         3
110413         3
110465         2
110564         1
110813         1
              ..
W./C. 6608     4
W./C. 6609     1
W.E.P. 5734    1
W/C 14208      1
WE/P 5735      2
Length: 681, dtype: int64

In [67]:
df.groupby("Cabin").size()

Cabin
A10    1
A14    1
A16    1
A19    1
A20    1
      ..
F33    3
F38    1
F4     2
G6     4
T      1
Length: 147, dtype: int64

In [68]:
df.groupby("Embarked").size()

Embarked
C    168
Q     77
S    644
dtype: int64

based on the grouping above, it seems like ticket and cabins are unique (too long for me to care), so i will use the embarked, and fill the nan value with mode (based on class)

In [69]:
# drop cabin and ticket
df.drop("Cabin", axis=1, inplace=True)
df.drop("Ticket", axis=1, inplace=True)

In [70]:
# change embarked to numeric
def numeric_embarked(df):
    df["Embarked"].replace('C', 1, inplace=True)
    df["Embarked"].replace('Q', 2, inplace=True)
    df["Embarked"].replace('S', 3, inplace=True)
    return df

#### Apply the preprocessing

In [71]:
df = remove_name(df)
df = numeric_sex(df)
df = numeric_embarked(df)

In [72]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,3.0
1,1,1,0,38.0,1,0,71.2833,1.0
2,1,3,0,26.0,0,0,7.9250,3.0
3,1,1,0,35.0,1,0,53.1000,3.0
4,0,3,1,35.0,0,0,8.0500,3.0


### Preprocessing pt.2
- outlier detection
- fill NA (age, embarked)
- split the x and y
- maybe some scaling (to normalize age, fare, pclass, sibsp)
- fix the imbalance

#### Outlier detection

In [73]:
boxplot = df.boxplot()

fare has too many outlier, tbh its better to drop it

In [74]:
df.drop("Fare", axis=1, inplace=True)

#### FillNA

In [75]:
# find the average age per class
average_age_class = df.groupby('Pclass')['Age'].mean()
# find the mode embarked per class
mode_embarked_class = df.groupby('Pclass')['Embarked'].agg(lambda x: x.mode().iloc[0])

# change into list
average_age_class = list(average_age_class)
mode_embarked_class = list(mode_embarked_class)

print(average_age_class)
print(mode_embarked_class)

[38.233440860215055, 29.87763005780347, 25.14061971830986]
[3.0, 3.0, 3.0]


In [76]:
# now fill the nan value
# first i'm splitting the dataset into 3 based on the Pclass
# df.groupby("Pclass").size() #-> to know how many Pclass in the dataset (1, 2, 3)
df1 = df.loc[df["Pclass"]==1]
df2 = df.loc[df["Pclass"]==2]
df3 = df.loc[df["Pclass"]==3]

# this function will fill the null embarked and age
def fill_null(df, age, column):
  df.loc[df["Age"].isna(), "Age"] = age
  df.loc[df["Embarked"].isna(), "Embarked"] = column
  return df

In [77]:
# fill the nan value
df1 = fill_null(df1, int(average_age_class[0]), int(mode_embarked_class[0]))
df2 = fill_null(df2, int(average_age_class[1]), int(mode_embarked_class[1]))
df3 = fill_null(df3, int(average_age_class[2]), int(mode_embarked_class[2]))

# combine and shuffle the dataset (i probably dont have to shuffle it but.. i may wrong)
final_df = pd.concat([df1, df2, df3])
final_df = final_df.sample(frac=1, random_state=64)

# cek nan value
final_df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

#### Splitting the x and y

In [78]:
labels = list(final_df.columns)
x = final_df[labels[1:]]
y = final_df[labels[0]]


#### Scaling

In [79]:
# pake scaling biar rata datanya
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
scaler.transform(x)

array([[-1.56610693, -1.35557354,  1.04110263, -0.4745452 ,  0.76762988,
         0.58595414],
       [ 0.82737724, -1.35557354, -0.32206462, -0.4745452 ,  2.00893337,
        -0.67817453],
       [-0.36936484,  0.73769513, -0.01913857, -0.4745452 , -0.47367361,
         0.58595414],
       ...,
       [-1.56610693,  0.73769513,  0.13232446,  0.43279337, -0.47367361,
         0.58595414],
       [-1.56610693,  0.73769513,  0.51098203, -0.4745452 , -0.47367361,
         0.58595414],
       [ 0.82737724, -1.35557354,  0.28378749,  2.24747049, -0.47367361,
         0.58595414]])

#### Fix imbalance

In [86]:
print("before sampling:")
test = y.to_frame()
test.groupby("Survived").size()

before sampling:


Survived
0    549
1    342
dtype: int64

In [80]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x, y)

In [84]:
test = y_train.to_frame()
test.groupby("Survived").size()

Survived
0    549
1    549
dtype: int64

### Model creation
The goal of this project is to determine wether the passenger is alive or not (binary classification) hence:
- logistic regression = good
- random forest = good

#### Logistic regression

In [87]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [92]:
df_test = pd.read_csv(test_path)
passenger_id = df_test["PassengerId"]

In [93]:
df_test.drop("PassengerId", axis=1, inplace=True)
df_test.drop("Name", axis=1, inplace=True)
df_test.drop("Fare", axis=1, inplace=True)
df_test.drop("Cabin", axis=1, inplace=True)
df_test.drop("Ticket", axis=1, inplace=True)
df_test = numeric_sex(df_test)
df_test = numeric_embarked(df_test)
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,1,34.5,0,0,2
1,3,0,47.0,1,0,3
2,2,1,62.0,0,0,2
3,3,1,27.0,0,0,3
4,3,0,22.0,1,1,3


In [95]:
scaler.transform(df_test)

array([[ 0.82737724,  0.73769513,  0.39738476, -0.4745452 , -0.47367361,
        -0.67817453],
       [ 0.82737724, -1.35557354,  1.34402868,  0.43279337, -0.47367361,
         0.58595414],
       [-0.36936484,  0.73769513,  2.48000139, -0.4745452 , -0.47367361,
        -0.67817453],
       ...,
       [ 0.82737724,  0.73769513,  0.70031081, -0.4745452 , -0.47367361,
         0.58595414],
       [ 0.82737724,  0.73769513,         nan, -0.4745452 , -0.47367361,
         0.58595414],
       [ 0.82737724,  0.73769513,         nan,  0.43279337,  0.76762988,
        -1.9423032 ]])

In [96]:
Survived = model.predict(df_test)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [131]:
passenger_id = list(passenger_id)
Survived = list(Survived)
print(passenger_id)
print(Survived)


[892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 107

In [132]:
datas = {
    "PassengerId" : passenger_id,
    "Survived" : Survived
}


In [135]:
ans = pd.DataFrame(datas)
ans.head() 

TypeError: Index(...) must be called with a collection of some kind, False was passed

In [136]:
ans.to_csv("prediction.csv", index=False)